In [ ]:
# Ad: Hilmi Tunahan
# Soyad: Başar
# Okul Numara: 2112721019
# GitHub Repo Bağlantısı: https://github.com/tunaBasar/MachineLearning.git

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle

df = pd.read_csv('data/shark_data.csv')

print("Veri Seti İlk 5 Satır:")
print(df.head())

print("\nVeri Bilgisi:")
print(df.info())

Veri Seti İlk 5 Satır:
       Tag_ID      Species  Gender   Length_cm  Fin_Height_cm  Color_Code  \
0  SHARK-9055        Tiger    Male  336.833472      59.045354           2   
1  SHARK-1313        Tiger  Female  466.130735      36.469152           1   
2  SHARK-9158  Great White  Female  538.180117      51.204832           1   
3  SHARK-3293        Tiger  Female  247.456339      52.319600           2   
4  SHARK-9228        Tiger  Female  466.315468      39.669043           4   

    Age    Weight_kg  
0  18.0   786.366159  
1  26.0   993.452824  
2  16.0  1110.600369  
3  27.0   583.292854  
4  37.0  1005.135981  

Veri Bilgisi:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Tag_ID         300 non-null    object 
 1   Species        300 non-null    object 
 2   Gender         300 non-null    object 
 3   Length_cm      300 non-null    f

In [5]:
print("Eksik Veri Sayıları:\n", df.isnull().sum())

df['Age'] = df['Age'].fillna(df['Age'].mean())

print("\nDoldurma İşlemi Sonrası Kontrol:\n", df.isnull().sum())

Eksik Veri Sayıları:
 Tag_ID            0
Species           0
Gender            0
Length_cm         0
Fin_Height_cm     0
Color_Code        0
Age              25
Weight_kg         0
dtype: int64

Doldurma İşlemi Sonrası Kontrol:
 Tag_ID           0
Species          0
Gender           0
Length_cm        0
Fin_Height_cm    0
Color_Code       0
Age              0
Weight_kg        0
dtype: int64


In [6]:
df = df.drop(['Tag_ID'], axis=1)
print("Tag_ID sütunu çıkarıldı.")

Tag_ID sütunu çıkarıldı.


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])

df = pd.get_dummies(df, columns=['Species'], drop_first=True)

df = df.astype(float)

print("Encoding Sonrası Veri Seti:")
print(df.head())

Encoding Sonrası Veri Seti:
   Gender   Length_cm  Fin_Height_cm  Color_Code   Age    Weight_kg  \
0     1.0  336.833472      59.045354         2.0  18.0   786.366159   
1     0.0  466.130735      36.469152         1.0  26.0   993.452824   
2     0.0  538.180117      51.204832         1.0  16.0  1110.600369   
3     0.0  247.456339      52.319600         2.0  27.0   583.292854   
4     0.0  466.315468      39.669043         4.0  37.0  1005.135981   

   Species_Hammerhead  Species_Tiger  
0                 0.0            1.0  
1                 0.0            1.0  
2                 0.0            0.0  
3                 0.0            1.0  
4                 0.0            1.0  


In [8]:
X = df.drop('Weight_kg', axis=1)
y = df['Weight_kg']

X_opt = sm.add_constant(X)

model_ols = sm.OLS(y, X_opt).fit()
print(model_ols.summary())


                            OLS Regression Results                            
Dep. Variable:              Weight_kg   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     2684.
Date:                Sun, 14 Dec 2025   Prob (F-statistic):          7.56e-261
Time:                        20:57:54   Log-Likelihood:                -1438.7
No. Observations:                 300   AIC:                             2893.
Df Residuals:                     292   BIC:                             2923.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                -15.1214      9

In [9]:


if 'Color_Code' in X.columns:
    X = X.drop('Color_Code', axis=1)
    print("Color_Code sütunu modelden çıkarıldı (p-value > 0.05).")

X_opt = sm.add_constant(X)
model_final_ols = sm.OLS(y, X_opt).fit()
print(model_final_ols.summary())

Color_Code sütunu modelden çıkarıldı (p-value > 0.05).
                            OLS Regression Results                            
Dep. Variable:              Weight_kg   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     3120.
Date:                Sun, 14 Dec 2025   Prob (F-statistic):          3.49e-262
Time:                        20:57:57   Log-Likelihood:                -1439.7
No. Observations:                 300   AIC:                             2893.
Df Residuals:                     293   BIC:                             2919.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"R-Kare (R2) Skoru: {r2:.4f}")
print(f"MAE (Ortalama Mutlak Hata): {mae:.2f}")
print(f"MSE (Ortalama Kare Hata): {mse:.2f}")
print(f"RMSE (Kök Ortalama Kare Hata): {rmse:.2f}")


R-Kare (R2) Skoru: 0.9815
MAE (Ortalama Mutlak Hata): 26.79
MSE (Ortalama Kare Hata): 1050.42
RMSE (Kök Ortalama Kare Hata): 32.41


In [11]:
pickle.dump(regressor, open('model.pkl', 'wb'))
print("Model başarıyla 'model.pkl' olarak kaydedildi.")

pickle.dump(X.columns.tolist(), open('features.pkl', 'wb'))
print("Feature isimleri 'features.pkl' olarak kaydedildi.")

Model başarıyla 'model.pkl' olarak kaydedildi.
Feature isimleri 'features.pkl' olarak kaydedildi.


In [13]:
import pandas as pd
import pickle


loaded_model = pickle.load(open('model.pkl', 'rb'))

beklenen_sutunlar = X.columns.tolist()
print(f"Modelin Beklediği Sütunlar: {beklenen_sutunlar}")

yeni_kopekbaligi = pd.DataFrame(0, index=[0], columns=beklenen_sutunlar)

yeni_kopekbaligi['Length_cm'] = 400
yeni_kopekbaligi['Fin_Height_cm'] = 60
yeni_kopekbaligi['Age'] = 15
yeni_kopekbaligi['Gender'] = 1

if 'Species_Tiger' in yeni_kopekbaligi.columns:
    yeni_kopekbaligi['Species_Tiger'] = 1
tahmin = loaded_model.predict(yeni_kopekbaligi)

print(f"\nBu özelliklerdeki köpekbalığının tahmini ağırlığı: {tahmin[0]:.2f} kg")

Modelin Beklediği Sütunlar: ['Gender', 'Length_cm', 'Fin_Height_cm', 'Age', 'Species_Hammerhead', 'Species_Tiger']

Bu özelliklerdeki köpekbalığının tahmini ağırlığı: 885.41 kg
